In [5]:
%reload_ext autoreload
%autoreload 2
try:
    import notebook_setup
except ImportError:
    print('Could not import notebook setup file (not critical).')

import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

from bdx import get_trend
try:
    from utils import get_credentials
    username, password = get_credentials()
except ImportError:
    print('Manually specify username and password')
    username, password = None, None

cst = timezone(offset=-timedelta(hours=6))
cdt = timezone(offset=-timedelta(hours=5))

trend = '3481'      # python setpoint trend
ch2trend = '2841'  # chiller 2 detailed trend
ch1trend = '2422'  # chiller 1 detailed trend

ch1start, ch1end = datetime(2021,4,1, tzinfo=cdt), datetime(2021,8,2, tzinfo=cdt)
ch2start, ch2end = datetime(2021,4,1, tzinfo=cdt), datetime(2021,8,2, tzinfo=cdt)

In [2]:
from controllers.esb import get_controller, get_current_state
from controller import 

In [6]:
get_controller(stepsize=1, target='temperature', bounds=((55,75),), window=1, tolerance=0.05)

Controller(bounds=array([[55, 75]]), stepsize=1, target='temperature',
           tolerance=0.05, window=1)

## Chiller 1

In [2]:
data1 = get_trend(ch1trend, username, password,
                    start=ch1start,
                    end=ch1end)

In [3]:
setpoint = get_trend(trend, username, password,
                    start=ch1start,
                    end=ch1end)
data1['Setpoint'] = setpoint['CDWTPythonSetpt']

In [ ]:
data1.describe()

In [ ]:
data1.PerFreqFanA.plot()

## Chiller 2

In [ ]:
data2 = get_trend(ch2trend, username, password,
                    start=ch2start,
                    end=ch2end)

In [ ]:
data2.describe()

In [ ]:
data2.PerFreqFanA.plot()

In [ ]:
plt.hist(data2.PerFreqFanA, density=False)

## RL Controller

In [4]:
# Features for the data-model for environment
# State + action variables
colsx = [
    # Ambient
    'TempWetBulb', 'TempAmbient',
    # Machine temperatures
    'TempCondOut',
    # Machine state
    'Tonnage', 'PressDiffCond',
    # Action
    'Setpoint'
]
state_vars = colsx[:-1]
# Variables for cooling tower conditions that are staged
ticker_vars = ['TempWetBulb', 'TempAmbient', 'Tonnage', 'PressDiffCond']
lag = 1
colsy = ['TempCondIn', 'TempCondOut', 'PowFanA']

In [5]:
def get_env_data(df, colsx, colsy, ticker_vars, lag=1, train_split=0.9):
    data = df.loc[:, list(set(colsx+colsy+ticker_vars))]
    data = data.dropna(axis=0, how='any')
    ticker = [day_data for date, day_data in data[ticker_vars].groupby(data.index.date)]
    ticker = [t for t in ticker if len(t)==288] # all samples for a day
    x = data.loc[data.index[:-lag], colsx]
    y = data.loc[data.index[lag:], colsy]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    x = scaler.fit_transform(x)
    y = y.to_numpy().squeeze()

    x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=train_split)
    return x_train, x_val, y_train, y_val, ticker, scaler

def train_model(x, y, **model_params):
    params = dict(hidden_layer_sizes=(32,32,32), learning_rate_init=1e-3, max_iter=500, verbose=True)
    params.update(model_params)
    model = MLPRegressor(**params)
    model.fit(x, y)
    return model

def get_env(model_fn, scaler_fn, ticker, seed=0):
    if isinstance(model_fn, sklearn.base.BaseEstimator):
        model_fn = model_fn.predict
    if isinstance(scaler_fn, sklearn.base.TransformerMixin):
        scaler_fn = scaler_fn.transform
    return CoolingTowerEnv(model_fn, ticker, seed, scaler_fn)

In [ ]:
x_train1, x_val1, y_train1, y_val1, ticker1, scaler1 = get_env_data(data1, colsx, colsy, ticker_vars)
model1 = train_model(x_train1, y_train1)

In [9]:
from systems import CoolingTowerEnv
from controllers.esb_rl import get_controller, get_current_state, update_controller

In [10]:
ctrl = get_controller()

In [ ]:
for index, state in data1.iloc[:50].iterrows():
    action, = ctrl.predict(state)
    update_controller(ctrl, update_interval=10)

In [ ]:
ctrl.memory.states